# 7.6 Randomly generated parameters

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


When you're testing out a model, especially in the early stages of development, you
may find it convenient to let randomly generated data stand in for actual data to be
obtained later. Randomly generated parameters can also be useful in experimenting with
alternative model formulations or solvers.

Randomly generated parameters are like the computed parameters introduced in the
preceding section, except that their defining expressions are made random by use of
AMPL's built-in random number generation functions listed in [Table A-3](../appendix/appendix.ipynb#table-A-3) xTODO. As an example
of the simplest case, the individual parameter avail representing hours available in
`steel.mod` may be defined to equal a random function:
```
param avail_mean > 0;
param avail_variance > 0, < avail_mean / 2;
param avail = max(Normal(avail_mean, avail_variance), 0);
```
Adding some indexing gives a multi-stage version of this model:
```
param avail {STAGE} = max(Normal(avail_mean, avail_variance), 0);
```
For each stage `s`, this gives `avail[s]` a different random value from the same random
distribution. To specify stage-dependent random distributions, you would add indexing
to the mean and variance parameters as well:
```
param avail_mean {STAGE} > 0;
param avail_variance {s in STAGE} > 0, < avail_mean[s] / 2;
param avail {s in STAGE} = max(Normal(avail_mean[s], avail_variance[s]), 0);
```
The `max(...,0)` expression is included to handle the rare case in which the normal distribution
with a positive mean returns a negative value.

More general ways of randomly computing parameters arise naturally from the preceding
section's examples. In the multicommodity transportation problem, you can
define random shares of demand:
```
param share {DEST} = Uniform(0,100);
param tot_sh = sum {j in DEST} share[j];
param tot_dem {PROD} >= 0;
param demand {j in DEST, p in PROD}
       = share[j] * tot_dem[p] / tot_sh;
```

Parameters `tot_sh` and demand then also become random, because they are defined in
terms of random parameters. In the multiperiod production model, you can define the
demand quantities `market[p,t]` in terms of an initial value and a random amount of
increase per period:
```
param market1 {PROD} >= 0;
param max_incr {PROD} >= 0;
param market {p in PROD, t in 1..T+1} =
  if t = 1 then market1[p]
     else Uniform(0,max_incr) * market[p,t-1];
```
A recursive definition of this kind provides a way of generating simple random processes
over time.

All of the AMPL random functions are based on a uniform random number generator
with a very long period. When you start AMPL or give a reset command, however, the
generator is reset and the "random" values are the same as before. You can request different
values by changing the AMPL option `randseed` to some integer other than its
default value of 1; the command for this purpose is
```
ampl.option['randseed'] = n
```
where `n` is some integer value. Nonzero values give sequences that repeat each time
AMPL is reset. A value of 0 requests AMPL to pick a seed based on the current value of
the system clock, resulting (for practical purposes) in a different seed at each reset.

AMPL's reset data command, when applied to a randomly computed parameter,
also causes a new sample of random values to be determined. The use of this command
is discussed in Section 11.3. xTODO